This code is for generating fake choice data and recovering generative parameters

In [1]:
addprocs(48);

In [2]:
using PyPlot, StatsBase, JLD, Pandas, DataFrames, Distributions
@everywhere using choice_observation, latent_DDM_common_functions, helpers

In [21]:
dt,n,map_str,N,c,ntrials = 2e-2,103,"exp",2,5.,Int(5e4)

(0.02, 103, "exp", 2, 5.0, 50000)

In [23]:
pz = DataFrames.DataFrame(generative = vcat(1e-6,10.,-0.5,40.,1.,1.,0.02), 
    name = vcat("σ_i","B", "λ", "σ_a","σ_s","ϕ","τ_ϕ"),
    fit = vcat(falses(1),trues(4),falses(2)),
    initial = vcat(1.,5.,-5,100.,2.,0.2,0.08));

pz[:initial][pz[:fit] .== false] = pz[:generative][pz[:fit] .== false];

In [24]:
pd = DataFrames.DataFrame(generative = 0.1, name = "bias", fit = true, initial = 0.);
pchoice = vcat(pz,pd);

In [22]:
data = latent_DDM_common_functions.sample_clicks(ntrials,dt);

In [25]:
choice_observation.sampled_dataset!(data,pchoice[:generative]; num_reps=1, rng=4);

In [26]:
choice_observation.do_LL(pchoice[:generative],dt,data,n)

-9752.956715366083

In [ ]:
pchoice[:final],opt_output,state = choice_observation.do_optim(copy(pchoice[:initial]),
    pchoice[:fit],dt,data,n;map_str=map_str);

Iter     Function value   Gradient norm 
     0     1.493047e+04     5.971926e+03
     1     1.254925e+04     1.020964e+04
     2     1.062708e+04     8.955435e+03
     3     1.043153e+04     2.880100e+03
     4     1.013358e+04     3.560553e+03
     5     1.010507e+04     1.165552e+03
     6     1.004257e+04     7.494641e+02
     7     1.003595e+04     4.283952e+03
     8     9.935242e+03     3.621544e+03
     9     9.848277e+03     1.456926e+03
    10     9.810400e+03     1.913728e+03
    11     9.766063e+03     9.014393e+02
    12     9.749736e+03     3.274701e+02
    13     9.740807e+03     2.979993e+02
    14     9.731892e+03     1.940202e+02
    15     9.730634e+03     7.133506e+00
    16     9.730610e+03     6.187127e+00
    17     9.730609e+03     6.191352e-01
    18     9.730609e+03     7.201787e-01
    19     9.730606e+03     2.615570e-01
    20     9.730606e+03     1.021763e-01
    21     9.730606e+03     1.530949e-01
    22     9.730606e+03     9.254504e-02
    23     9.730

In [ ]:
pchoice[:CI_z_plus], pchoice[:CI_z_minus], H = choice_observation.do_H(pchoice[:final],pchoice[:fit],
    dt,data,n;map_str=map_str)

pchoice[:within_bounds] = (pchoice[:CI_z_minus] .< pchoice[:generative]) .& (pchoice[:CI_z_plus] .> pchoice[:generative]);

In [ ]:
pchoice